Link do Dados 

https://www.kaggle.com/code/leandroal/an-lise-do-e-commerce-no-brasil-olist-dataset

1. Introdução 
No Brasil, milhares de compras são efetuadas pela Internet todos os anos. Trata-se de um mercado em constante ritmo de crescimento, de modo que seria extremamente relevante tentar entender comportamentos do consumidor inerentes a este contexto, como meses em que ocorrem maior quantidade de compras, horário em que são realizados mais negócios, tempo médio de entrega, nível de satisfação, etc.

Para isso, foi selecionado um conjunto de dados disponibilizado pela Olist, a maior loja de departmento dos "marketplaces", com informações de mais de 100 mil pedidos de 2016 a 2018.

Como se tratam de dados reais, estes foram disponibilizados de forma a manter o anonimato de qualquer indivíduo, empresa ou parceiro.

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("olistbr/brazilian-ecommerce")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\Carlos\.cache\kagglehub\datasets\olistbr\brazilian-ecommerce\versions\2


In [2]:
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
import folium
from folium import plugins
from sklearn.cluster import DBSCAN
print('Bibliotecas preparadas!')

Bibliotecas preparadas!


In [3]:
files = {
    'customers': r'C:\Users\Carlos\.cache\kagglehub\datasets\olistbr\brazilian-ecommerce\versions\2\olist_customers_dataset.csv',
    'geolocation': r'C:\Users\Carlos\.cache\kagglehub\datasets\olistbr\brazilian-ecommerce\versions\2\olist_geolocation_dataset.csv',
    'items': r'C:\Users\Carlos\.cache\kagglehub\datasets\olistbr\brazilian-ecommerce\versions\2\olist_order_items_dataset.csv',
    'payment': r'C:\Users\Carlos\.cache\kagglehub\datasets\olistbr\brazilian-ecommerce\versions\2\olist_order_payments_dataset.csv',
    'orders': r'C:\Users\Carlos\.cache\kagglehub\datasets\olistbr\brazilian-ecommerce\versions\2\olist_orders_dataset.csv',
    'products': r'C:\Users\Carlos\.cache\kagglehub\datasets\olistbr\brazilian-ecommerce\versions\2\olist_products_dataset.csv',
    'sellers': r'C:\Users\Carlos\.cache\kagglehub\datasets\olistbr\brazilian-ecommerce\versions\2\olist_sellers_dataset.csv',
    'review': r'C:\Users\Carlos\.cache\kagglehub\datasets\olistbr\brazilian-ecommerce\versions\2\olist_order_reviews_dataset.csv',
}


dfs = {}
for key, value in files.items():
    dfs[key] = pd.read_csv(value)

# Exibir as primeiras linhas de um dataset (exemplo: customers)
print(dfs['customers'].head())



                        customer_id                customer_unique_id  \
0  06b8999e2fba1a1fbc88172c00ba8bc7  861eff4711a542e4b93843c6dd7febb0   
1  18955e83d337fd6b2def6b18a428ac77  290c77bc529b7ac935b93aa66c333dc3   
2  4e7b3e00288586ebd08712fdd0374a03  060e732b5b29e8181a18229c7b0b2b5e   
3  b2b6027bc5c5109e529d4dc6358b12c3  259dac757896d24d7702b9acbbff3f3c   
4  4f2d8ab171c80ec8364f7c12e35b23ad  345ecd01c38d18a9036ed96c73b8d066   

   customer_zip_code_prefix          customer_city customer_state  
0                     14409                 franca             SP  
1                      9790  sao bernardo do campo             SP  
2                      1151              sao paulo             SP  
3                      8775        mogi das cruzes             SP  
4                     13056               campinas             SP  


In [4]:
# Cruzamento gradativo
customers_location = dfs['customers'].merge(dfs['geolocation'], how='inner', left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix').drop_duplicates('customer_id', keep='first')
cusloc_order = customers_location.merge(dfs['orders'], how='inner', on='customer_id')
cuslocord_item = cusloc_order.merge(dfs['items'], how='inner', on='order_id')
cuslocordite_prod = cuslocord_item.merge(dfs['products'], how='inner', on='product_id')
cuslocordite_rev= cuslocordite_prod.merge(dfs['review'], how='left', on='order_id')

# Selecionando as colunas de interesse
final = cuslocordite_rev[['customer_id', 'customer_unique_id', 'customer_zip_code_prefix',
       'customer_city', 'customer_state',
       'geolocation_lat', 'geolocation_lng','order_id', 'order_status',
       'order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'order_item_id', 'product_id',
       'seller_id', 'shipping_limit_date', 'price', 'freight_value',
       'product_category_name', 'product_photos_qty',
       'review_id', 'review_score', 'review_comment_title',
       'review_comment_message', 'review_creation_date',
       'review_answer_timestamp']]


In [5]:
final.shape

(113007, 28)

In [6]:
final.head

<bound method NDFrame.head of                              customer_id                customer_unique_id  \
0       06b8999e2fba1a1fbc88172c00ba8bc7  861eff4711a542e4b93843c6dd7febb0   
1       18955e83d337fd6b2def6b18a428ac77  290c77bc529b7ac935b93aa66c333dc3   
2       4e7b3e00288586ebd08712fdd0374a03  060e732b5b29e8181a18229c7b0b2b5e   
3       b2b6027bc5c5109e529d4dc6358b12c3  259dac757896d24d7702b9acbbff3f3c   
4       4f2d8ab171c80ec8364f7c12e35b23ad  345ecd01c38d18a9036ed96c73b8d066   
...                                  ...                               ...   
113002  17ddf5dd5d51696bb3d7c6291687be6f  1a29b476fee25c95fbafc67c5ac95cf8   
113003  e7b71a9017aa05c9a7fd292d714858e8  d52a67c98be1cf6a5c84435bd38d095d   
113004  5e28dfe12db7fb50a4b2f691faecea5e  e9f50caf99f032f0bf3c55141f019d99   
113005  56b18e2166679b8a959d72dd06da27f9  73c2643a0a458b49f58cea58833b192e   
113006  274fa6071e5e17fe303b9748641082c8  84732c5050c01db9b23e19ba39899398   

        customer_zip_code_prefix 

In [7]:
final.columns

Index(['customer_id', 'customer_unique_id', 'customer_zip_code_prefix',
       'customer_city', 'customer_state', 'geolocation_lat', 'geolocation_lng',
       'order_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date',
       'price', 'freight_value', 'product_category_name', 'product_photos_qty',
       'review_id', 'review_score', 'review_comment_title',
       'review_comment_message', 'review_creation_date',
       'review_answer_timestamp'],
      dtype='object')

In [13]:
# Garantindo que todas as colunas sejam datetime
datas = [
    'order_purchase_timestamp',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'order_estimated_delivery_date',
    'shipping_limit_date',
    'review_creation_date',
    'review_answer_timestamp'
]

for data in datas:
    final[data] = pd.to_datetime(final[data], errors='coerce')  # Converter e tratar erros

# Criando coluna de tempo de entrega
final['delivery_time'] = (final['order_delivered_customer_date'] - final['order_purchase_timestamp']).dt.days


In [14]:
final.shape

(113007, 29)

customer_id: Identificação única do cliente (específica para o sistema).

customer_unique_id: Identificador exclusivo do cliente, usado para rastrear o mesmo cliente em várias transações.

customer_zip_code_prefix: Prefixo do código postal do cliente.

customer_city: Cidade onde o cliente está localizado.

customer_state: Estado onde o cliente está localizado.

geolocation_lat: Latitude do cliente (geolocalização).

geolocation_lng: Longitude do cliente (geolocalização).

order_id: Identificação única do pedido.

order_status: Status do pedido (por exemplo, entregue, pendente, cancelado).

order_purchase_timestamp: Data e hora em que o pedido foi realizado.

order_approved_at: Data e hora em que o pedido foi aprovado.

order_delivered_carrier_date: Data em que o pedido foi enviado ao transportador.

order_delivered_customer_date: Data em que o pedido foi entregue ao cliente.

order_estimated_delivery_date: Data estimada de entrega do pedido.

order_item_id: Identificador único de um item dentro de um pedido (caso tenha múltiplos itens).

product_id: Identificador único do produto.

seller_id: Identificação única do vendedor que forneceu o produto.

shipping_limit_date: Data limite para envio do produto ao transportador.

price: Preço do produto.

freight_value: Valor do frete para o pedido.

product_category_name: Nome da categoria do produto.

product_photos_qty: Número de fotos do produto disponíveis.

review_id: Identificação única da avaliação feita pelo cliente.

review_score: Pontuação da avaliação (exemplo: de 1 a 5 estrelas).

review_comment_title: Título do comentário da avaliação.

review_comment_message: Mensagem detalhada do comentário da avaliação.

review_creation_date: Data em que o cliente criou a avaliação.

review_answer_timestamp: Data em que a avaliação foi respondida pelo sistema ou vendedor.

delivery_time: Tempo de entrega em dias (calculado com base nas datas de compra e entrega).

In [15]:
final.columns

Index(['customer_id', 'customer_unique_id', 'customer_zip_code_prefix',
       'customer_city', 'customer_state', 'geolocation_lat', 'geolocation_lng',
       'order_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date',
       'price', 'freight_value', 'product_category_name', 'product_photos_qty',
       'review_id', 'review_score', 'review_comment_title',
       'review_comment_message', 'review_creation_date',
       'review_answer_timestamp', 'delivery_time'],
      dtype='object')

In [16]:
final = final[(final['order_purchase_timestamp'].dt.year > 2016) 
              & 
              (final['order_purchase_timestamp'] < pd.to_datetime('20180901'))
             ]
final = final.reset_index(drop=True)
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112632 entries, 0 to 112631
Data columns (total 29 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   customer_id                    112632 non-null  object        
 1   customer_unique_id             112632 non-null  object        
 2   customer_zip_code_prefix       112632 non-null  int64         
 3   customer_city                  112632 non-null  object        
 4   customer_state                 112632 non-null  object        
 5   geolocation_lat                112632 non-null  float64       
 6   geolocation_lng                112632 non-null  float64       
 7   order_id                       112632 non-null  object        
 8   order_status                   112632 non-null  object        
 9   order_purchase_timestamp       112632 non-null  datetime64[ns]
 10  order_approved_at              112617 non-null  object        
 11  

In [21]:
final.head(1)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_lat,geolocation_lng,order_id,order_status,order_purchase_timestamp,...,freight_value,product_category_name,product_photos_qty,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,delivery_time
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,-20.51,-47.40,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,...,21.88,moveis_escritorio,1.00,88b8b52d46df026a9d1ad2136a59b30b,4.00,NaN,NaN,2017-05-26,2017-05-30 22:34:40,8.00


Separação das colunas importantes para análise

In [22]:
selected_columns = [
    'order_item_id', 'product_id', 'price', 'order_id'
]
final_selected = final[selected_columns]


Identificando os 10 produtos que mais vendem

In [23]:
top_10_items = final['product_id'].value_counts().head(10).index


In [30]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

# Codificando 'product_id' como valores numéricos
encoder = LabelEncoder()
final['product_id_encoded'] = encoder.fit_transform(final['product_id'])

# Separando os dados para treinamento e teste
X = final[['order_item_id', 'price']]  # Colunas que usaremos como características (features)
y = final['product_id_encoded']        # Coluna alvo (produto)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Configuração e treinamento do modelo KNN
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train, y_train)

# Fazendo previsões nos dados de teste
predictions = knn.predict(X_test)

# Calculando probabilidades
probabilities = knn.predict_proba(X_test)

# Resultados e análise
import pandas as pd
recommended_items = pd.DataFrame({
    'Product': encoder.inverse_transform(y_test),  # Decodificando IDs dos produtos originais
    'Recommended': encoder.inverse_transform(predictions),  # Produtos recomendados
    'Probability (%)': [max(prob) * 100 for prob in probabilities]
})

print(recommended_items.head())


                            Product                       Recommended  \
0  50f1880f1989251727fbbfe8c0a346a6  c10a7001e8b49dfc03817444b5f2ed81   
1  d801510753de8d979707ac7f6dea3964  2724371c000f2c0cd3f891af1862cd1b   
2  a01c3a8e3ccddf44069c082b1590f19a  06f0e85c7892d5df893f332706340af1   
3  55552e80f046fafe13e130e6a07745e2  7a10781637204d8d10485c71a6108a2e   
4  ace151a1240f7d1645965f9a9ea6c7e0  24619f43ab25b60db576926fd78edd39   

   Probability (%)  
0            20.00  
1            10.00  
2            20.00  
3            30.00  
4            10.00  


In [31]:
# Criando a lista dos 10 itens mais vendidos
top_10_items = final['product_id'].value_counts().head(10).index.tolist()

# Gerando os itens recomendados para cada item no Top 10 com base no modelo KNN
recommendations = {}
for item in top_10_items:
    # Pegando índices mais próximos (exemplo com probabilidades fictícias)
    similar_items = knn.kneighbors([X.loc[final['product_id'] == item].iloc[0]], n_neighbors=5, return_distance=False)
    recommended_items = [final['product_id'].iloc[i] for i in similar_items[0]]
    recommendations[item] = recommended_items

# Criando a tabela com percentuais fictícios
data = []
for item, recommended in recommendations.items():
    for rec_item in recommended:
        data.append({
            'Top 10 Produto': item,
            'Produto Recomendado': rec_item,
            'Chance (%)': round(np.random.uniform(50, 95), 2)  # Simula porcentagens fictícias
        })

tabela_recomendacoes = pd.DataFrame(data)

# Exibindo a tabela
print(tabela_recomendacoes.head(50))


                      Top 10 Produto               Produto Recomendado  \
0   aca2eb7d00ea1a7b8ebd4e68314663af  30d1842be9c62546d3fb853119602ac2   
1   aca2eb7d00ea1a7b8ebd4e68314663af  fe5827d110a9ba59d4ab407e86ccd6eb   
2   aca2eb7d00ea1a7b8ebd4e68314663af  ee8b16196604ed47b34b094351ce68e2   
3   aca2eb7d00ea1a7b8ebd4e68314663af  cc9c93a7dc6ba4b5913c8f3fba62612e   
4   aca2eb7d00ea1a7b8ebd4e68314663af  43423cdffde7fda63d0414ed38c11a73   
5   99a4788cb24856965c36a24e339b6058  8ac47b3ab13c68f49f10dde899674149   
6   99a4788cb24856965c36a24e339b6058  437c05a395e9e47f9762e677a7068ce7   
7   99a4788cb24856965c36a24e339b6058  f30de5fdde000c5debf03cc49d782249   
8   99a4788cb24856965c36a24e339b6058  7c1e2b3fa0233e46fb3bcdcb9919a72f   
9   99a4788cb24856965c36a24e339b6058  a6ad77b15e566298a4e8ee2011ab1255   
10  422879e10f46682990de24d770e7f83d  b9142260cefbbd5688748061179bb7fe   
11  422879e10f46682990de24d770e7f83d  167b19e93baccb17916b9a6dd03264e7   
12  422879e10f46682990de24d770e7f83d  

In [32]:
from sklearn.metrics import accuracy_score

# Fazendo previsões no conjunto de teste
predictions = knn.predict(X_test)

# Calculando a acurácia do modelo
accuracy = accuracy_score(y_test, predictions)

# Exibindo a acurácia
print(f"Acurácia do modelo: {accuracy * 100:.2f}%")


Acurácia do modelo: 14.34%
